# timeseries
* 時系列データの前処理について書いていく。

# 差分
* 時系列データは1時点離れたデータとの差を用いることが多い。このようなデータを差分系列、階差系列と呼ぶ。

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

### サンプルデータの作成

In [2]:
# 時間のリストの効率的な作り方がわからない
start_time = datetime(2017, 11, 1, 10, 0, 0, 0)
time_list = []
for i in range(100):
    t = start_time + timedelta(seconds=i)
    time_list.append(t)
    
price = 50000
price_list = []
for _ in range(100):
    price += np.random.randint(-500,500)
    price_list.append(price)

In [3]:
df = pd.DataFrame(price_list,index=time_list,columns=["price"])
df.head(10)

,price
2017-11-01 10:00:00,50300
2017-11-01 10:00:01,50444
2017-11-01 10:00:02,50767
2017-11-01 10:00:03,51143
2017-11-01 10:00:04,51544
2017-11-01 10:00:05,51365
2017-11-01 10:00:06,50918
2017-11-01 10:00:07,50882
2017-11-01 10:00:08,50545
2017-11-01 10:00:09,50635


* 上記のような秒間データがあったとする。

### 差分の作成

* pd.DataFrame.diff()を用いることで、差分を取ることができる。

In [4]:
df["diff_1"] = df.diff()
df.head()

,price,diff_1
2017-11-01 10:00:00,50300,NaN
2017-11-01 10:00:01,50444,144.0
2017-11-01 10:00:02,50767,323.0
2017-11-01 10:00:03,51143,376.0
2017-11-01 10:00:04,51544,401.0


* periodsを指定することで、差分を取る間隔を変更することができる。

In [5]:
df["diff_3"] = df["price"].diff(periods=3)
df.head()

,price,diff_1,diff_3
2017-11-01 10:00:00,50300,NaN,NaN
2017-11-01 10:00:01,50444,144.0,NaN
2017-11-01 10:00:02,50767,323.0,NaN
2017-11-01 10:00:03,51143,376.0,843.0
2017-11-01 10:00:04,51544,401.0,1100.0


# 移動平均

## rolling
* rollingを用いると、移動平均のような、DataFrameを上からなぞりつつ、特定の行数分だけ着目し、計算するということができる。
* 下記は、3秒ごとの平均（移動平均）を求めている。
* `window` で秒数を指定することができるのは便利。
* なお、下記のように、`window` に時間を用いている場合は、オプションの `center` を用いることはできない。

In [6]:
df["rolling_mean_3s"] = df["price"].rolling(window="3s").mean()
df.head()

,price,diff_1,diff_3,rolling_mean_3s
2017-11-01 10:00:00,50300,NaN,NaN,50300.000000
2017-11-01 10:00:01,50444,144.0,NaN,50372.000000
2017-11-01 10:00:02,50767,323.0,NaN,50503.666667
2017-11-01 10:00:03,51143,376.0,843.0,50784.666667
2017-11-01 10:00:04,51544,401.0,1100.0,51151.333333


* 上記は、ある行から過去に遡って3秒の行の平均を求めている。
* `2017-11-01 10:00:02` の行であれば、 `2017-11-01 10:00:00`, `2017-11-01 10:00:01`, `2017-11-01 10:00:02` の行の `price` を足し、平均し、 `2017-11-01 10:00:02` の行の `rolling_mean_3s` の列に記載している。
* `2017-11-01 10:00:00`はその前2秒のデータがないためあるだけの（その行だけの）平均を求めている。